In [54]:
import pandas as pd
from nltk import sent_tokenize
# encoding 문제시, engine = 'python' 입력할 것
data = pd.read_excel('outputVer2.xlsx')
data

,Unnamed: 0,asin,item_model_no,product_name,review,review_date,score,user
0,0,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I have a professional Nikon camera purchased s...,"June 30, 2019",5.0 out of 5 stars,Arsenals
1,1,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I like this camera but it was not what i expec...,"July 4, 2019",5.0 out of 5 stars,Samira Alshaibani
2,2,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,Now is a good time to travel.I'm a novice phot...,"June 25, 2019",5.0 out of 5 stars,Vargas Laurence
3,3,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I'm not sure if it was just the camera that I ...,"July 20, 2019",3.0 out of 5 stars,Diane
4,4,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,Looks good. Ok for taking our holiday snaps an...,"July 15, 2019",5.0 out of 5 stars,shawn mayun
5,5,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,"Poor instructions,No clear way to turn off vid...","July 26, 2019",2.0 out of 5 stars,Her Babsiness
6,6,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,This camera is bought for a birthday gift for ...,"July 5, 2019",5.0 out of 5 stars,lilygallego
7,7,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I was actually surprised about this camera. Yo...,"August 2, 2019",5.0 out of 5 stars,Brandon
8,8,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,"I very like it, user friendly, easy operation,...","July 10, 2019",5.0 out of 5 stars,Elizebeth Thomas
9,9,B07S1HTSGH,NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,It is small，compact camera with required funct...,"July 25, 2019",5.0 out of 5 stars,Ling


In [2]:
len(data)

103327

In [55]:
import pandas as pd
from nltk import sent_tokenize
import re

def make_same_feature(text):
    
    # 각각의 feature 별 동의어를 대표명으로 정리
    text=re.sub('resolution|pixel|megapixel', 'pixel', text)
    text=re.sub('lens|wide\sangle|normal\srange', 'lens', text)
    text=re.sub('optical|zoom|optical\szoom|digital\szoom', 'optical', text)
    text=re.sub('memory|megabytes|MB', 'memory', text)
    text=re.sub('burst|continuous|shutter|recovery|motion|sport', 'burst', text)
    text=re.sub('battery|batteries|power', 'battery', text)
    text=re.sub('focus|exposure|manual|iso', 'focus', text)
    text=re.sub('LCD|screen', 'LCD', text)
    text=re.sub('compression|compress|jpeg', 'compression', text)
    text=re.sub('flash|light', 'flash', text)
    return text

def pre_processing(data):
    # score 변수를 정리하여 현재 점수만을 표현하도록 변경
    data['score'] = data['score'].apply(lambda x : x.split(' ')[0])
    
    result = []
    for i in range(len(data)):
        # 아래 for문에서 딕셔너리를 만드릭 위해 각각의 데이터를 변수로 저장
        asin = data.loc[i, 'asin']
        model = data.loc[i, 'item_model_no']
        product = data.loc[i, 'product_name']
        date = data.loc[i, 'review_date']
        score = data.loc[i, 'score']
        user = data.loc[i, 'user']
        text = data.loc[i, 'review']

        Sentences_list = sent_tokenize(text)
        for txt in Sentences_list:
            dict_n = {
                'product' : product,
                'asin' : asin,
                'model' : model,
                'date' : date,
                'score' : score,
                'user' : user,
                'review' : txt
            }
            result.append(dict_n)
    print(len(result))
    # 데이터프레임으로 형식 전환
    df = pd.DataFrame(result)
    
    # 각 feature별 동의어 처리 함수 사용
    df['review'] = df['review'].apply(make_same_feature)
    
    # feature가 들어있는 문장만 별로로 출력
    # str은 앞의 시리즈 데이터를 문자열로 처리하기 위한 변환식
    # contains는 데이터 내에 패턴이 존재하기만 하면 위치에 상관없이 True 값 출력
    df = df[df.review.str.contains('pixel|lens|optical|memory|burst|battery|focus|LCD|compression|flash') == True]
    df = df.reset_index(drop = True)
    return df

In [56]:
data_pp = pre_processing(data)
data_pp

353473


,asin,date,model,product,review,score,user
0,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I am so glad I found this inexpensive camera o...,5.0,Arsenals
1,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,The pictures came out very clear and very real...,5.0,Arsenals
2,B07S1HTSGH,"June 25, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,Now is a good time to travel.I'm a novice phot...,5.0,Vargas Laurence
3,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I'm not sure if it was just the camera that I ...,3.0,Diane
4,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,When I transferred the pictures and videos to ...,3.0,Diane
5,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,The camera is exactly like the picture and the...,3.0,Diane
6,B07S1HTSGH,"July 26, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,"Poor instructions,No clear way to turn off vid...",2.0,Her Babsiness
7,B079GDB7C7,"July 23, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,The lenses included in this bundle are all great.,5.0,Elizabeth Bonecher-Brenaman
8,B079GDB7C7,"July 23, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,It takes time to get used to the flash balance...,5.0,Elizabeth Bonecher-Brenaman
9,B079GDB7C7,"July 23, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,"The moon is with 500mm lens, focus setting w/2...",5.0,Elizabeth Bonecher-Brenaman


In [5]:
data_pp.to_csv('pre_processingVer5.csv')